### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Define all Tpch tables

In [1]:
table_names = ['lineitem','customer','orders','supplier','nation','region']
tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/'
tpch_tables = []
for table_name in table_names:
    tpch_tables.append(tpch_dir+table_name+'.tbl')
tpch_tables

['/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/lineitem.tbl',
 '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/customer.tbl',
 '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/orders.tbl',
 '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/supplier.tbl',
 '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/nation.tbl',
 '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/region.tbl']

Read Arrow table

In [2]:
import pyarrow as pa
from pyarrow import csv

In [3]:
pa.show_versions()

pyarrow version info
--------------------
Package kind: not indicated
Arrow C++ library version: 2.0.0
Arrow C++ compiler: GNU 7.5.0
Arrow C++ compiler flags: -fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/hkverma/miniconda3/envs/tendb/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/arrow-cpp-ext_1604401055323/work=/usr/local/src/conda/arrow-cpp-2.0.0 -fdebug-prefix-map=/home/hkverma/miniconda3/envs/tendb=/usr/local/src/conda-prefix -fdiagnostics-color=always -fuse-ld=gold -O3 -DNDEBUG
Arrow C++ git revision: 4d367ccc36657519870e6efafc537e5c287fe5f1
Arrow C++ git description: 


In [4]:
csv_options = pa.csv.ParseOptions(delimiter='|')
arrow_tpch_tables = []
for tpch_table in tpch_tables:
    arrow_tpch_tables.append(pa.csv.read_csv(input_file=tpch_table, parse_options=csv_options))
arrow_tpch_tables

[pyarrow.Table
 L_ORDERKEY: int64
 L_PARTKEY: int64
 L_SUPPKEY: int64
 L_LINENUMBER: int64
 L_QUANTITY: int64
 L_EXTENDEDPRICE: double
 L_DISCOUNT: double
 L_TAX: double
 L_RETURNFLAG: string
 L_LINESTATUS: string
 L_SHIPDATE: timestamp[s]
 L_COMMITDATE: timestamp[s]
 L_RECEIPTDATE: timestamp[s]
 L_SHIPINSTRUCT: string
 L_SHIPMODE: string
 L_COMMENT: string,
 pyarrow.Table
 C_CUSTKEY: int64
 C_NAME: string
 C_ADDRESS: string
 C_NATIONKEY: int64
 C_PHONE: string
 C_ACCTBAL: double
 C_MKTSEGMENT: string
 C_COMMENT: string,
 pyarrow.Table
 O_ORDERKEY: int64
 O_CUSTKEY: int64
 O_ORDERSTATUS: string
 O_TOTALPRICE: double
 O_ORDERDATE: timestamp[s]
 O_ORDERPRIORITY: string
 O_CLERK: string
 O_SHIPPRIORITY: int64
 O_COMMENT: string,
 pyarrow.Table
 S_SUPPKEY: int64
 S_NAME: string
 S_ADDRESS: string
 S_NATIONKEY: int64
 S_PHONE: string
 S_ACCTBAL: double
 S_COMMENT: string,
 pyarrow.Table
 N_NATIONKEY: int64
 N_NAME: string
 N_REGIONKEY: int64
 N_COMMENT: string,
 pyarrow.Table
 R_REGIONKEY: 

In [5]:
for i in range(len(arrow_tpch_tables)):
    print(table_names[i]," Rows=", arrow_tpch_tables[i].num_rows," Cols=", arrow_tpch_tables[i].num_columns)

lineitem  Rows= 6005  Cols= 16
customer  Rows= 150  Cols= 8
orders  Rows= 1500  Cols= 9
supplier  Rows= 10  Cols= 7
nation  Rows= 25  Cols= 4
region  Rows= 5  Cols= 3


TenDB sample commands

In [6]:
import tendb as ten

<frozen importlib._bootstrap>:228: RuntimeWarning: compiletime version 3.6 of module 'tendb._tendb' does not match runtime version 3.9
<frozen importlib._bootstrap>:228: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [7]:
tb = ten._tendb.CTenDB()

Added a new cache


In [8]:
tb.show_versions()

'0.0.2'

In [9]:
btable_names = [b'lineitem',b'customer',b'orders',b'supplier',b'nation',b'region']
for i in range(len(btable_names)):
    tb.add_table(btable_names[i], arrow_tpch_tables[i])

Added Table= b'lineitem'
Added Table= b'customer'
Added Table= b'orders'
Added Table= b'supplier'
Added Table= b'nation'
Added Table= b'region'


In [10]:
tb.info()

b'[region,customer,lineitem,orders,supplier,nation,]'

In [11]:
tb.query6()

 TPCH QUERY 6 
SELECT 
  SUM(L_EXTENDEDPRICE * L_DISCOUNT) AS REVENUE 
FROM 
  LINEITEM
WHERE
  L_SHIPDATE >= DATE '1997-01-01'
  AND L_SHIPDATE < DATE '1997-01-01' + INTERVAL '1' YEAR
  AND L_DISCOUNT BETWEEN 0.07 - 0.01 AND 0.07 + 0.01
  AND L_QUANTITY < 25;



125257.80519999999

In [12]:
for table in btable_names:
    result = tb.make_dtensor(table)
    print(table,"=",result)

b'lineitem' = 0
b'customer' = 0
b'orders' = 0
b'supplier' = 0
b'nation' = 0
b'region' = 0


In [13]:
tb.query5()

 
SELECT
	N_NAME,
	SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE
FROM
	CUSTOMER,
	ORDERS,
	LINEITEM,
	SUPPLIER,
	NATION,
	REGION
WHERE
	C_CUSTKEY = O_CUSTKEY
	AND L_ORDERKEY = O_ORDERKEY
	AND L_SUPPKEY = S_SUPPKEY
	AND C_NATIONKEY = S_NATIONKEY
	AND S_NATIONKEY = N_NATIONKEY
	AND N_REGIONKEY = R_REGIONKEY
	AND R_NAME = 'EUROPE'
	AND O_ORDERDATE >= DATE '1995-01-01'
	AND O_ORDERDATE < DATE '1995-01-01' + INTERVAL '1' YEAR
GROUP BY
	N_NAME
ORDER BY
	REVENUE DESC;

b'UNITED KINGDOM' = 2132282.584500001


{b'UNITED KINGDOM': 2132282.584500001}